## PROMPT
Generate a pipeline for Databricks:

Get Data information from two web addresses:

Space launches: https://api.spacexdata.com/v3/launches

Space rockets: https://api.spacexdata.com/v3/rockets

Combine each launch record with the name of the corresponding rocket.

Send the final result (launches with rocket names) to the web address: https://httpbin.org/post

The script must provide status updates on its progress, report any errors encountered, confirm the outcome of the final data sending step, and measure/report execution times.

In [0]:
import requests
import json
from datetime import datetime
import time

# Function to get data from a URL
def get_data(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors
        return response.json()
    except requests.exceptions.RequestException as err:
        print(f"Error fetching data: {err}")
        return None

# Function to post data to a URL
def post_data(url, data):
    try:
        response = requests.post(url, json=data)
        response.raise_for_status()  # Raise an exception for HTTP errors
        return response.json()
    except requests.exceptions.RequestException as err:
        print(f"Error posting data: {err}")
        return None

# Main function to execute the pipeline
def execute_pipeline():
    start_time = time.time()
    print(f"Pipeline started at: {datetime.now()}")
    
    # Get space launches data
    launches_url = "https://api.spacexdata.com/v3/launches"
    print(f"Fetching launches data from: {launches_url}")
    launches_data = get_data(launches_url)
    if launches_data is None:
        print("Failed to fetch launches data. Exiting pipeline.")
        return
    
    # Get space rockets data
    rockets_url = "https://api.spacexdata.com/v3/rockets"
    print(f"Fetching rockets data from: {rockets_url}")
    rockets_data = get_data(rockets_url)
    if rockets_data is None:
        print("Failed to fetch rockets data. Exiting pipeline.")
        return
    
    # Combine each launch record with the name of the corresponding rocket
    print("Combining launches data with rocket names")
    combined_data = []
    for launch in launches_data:
        rocket_id = launch['rocket']['rocket_id']
        rocket_name = next((rocket['rocket_name'] for rocket in rockets_data if rocket['rocket_id'] == rocket_id), None)
        if rocket_name:
            launch['rocket_name'] = rocket_name
            combined_data.append(launch)
        else:
            print(f"Rocket name not found for launch with rocket ID: {rocket_id}")
    
    # Send the final result to the web address
    post_url = "https://httpbin.org/post"
    print(f"Posting combined data to: {post_url}")
    post_response = post_data(post_url, combined_data)
    if post_response is not None:
        print(f"Data posted successfully. Response: {post_response}")
    else:
        print("Failed to post data.")
    
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Pipeline finished at: {datetime.now()}")
    print(f"Execution time: {execution_time} seconds")

# Execute the pipeline
execute_pipeline()
